In [6]:
import torch
from get_scaleup import build_vssm
from vmamba.vmamba import VSSM, SS2D
from functools import partial
import os
from torchvision import transforms

data = "/dataset/ImageNet2012/"
def build_dataset(root=data, img_size=224):
    root = os.path.join(root, "val")
    transform = transforms.Compose([
        transforms.Resize(256/224 * img_size, interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    ])
    dataset = datasets.ImageFolder(root, transform=transform)
    return dataset


def add_hook(model: VSSM):

    def ss2d_forward(self: SS2D, x: torch.Tensor = None):
        if True:
            B, H, W, C = x.shape

            xz = self.in_proj(x)
            x, z = xz.chunk(2, dim=-1) # (b, h, w, d)

            x = x.permute(0, 3, 1, 2).contiguous()
            x = self.act(self.conv2d(x)) # (b, d, h, w)
            y1, y2, y3, y4 = self.forward_core(x)
            assert y1.dtype == torch.float32
            
            setattr(self, "y1", y1)
            setattr(self, "y2", y2)
            setattr(self, "y3", y3)
            setattr(self, "y4", y4)
            y = y1 + y2 + y3 + y4
            y = torch.transpose(y, dim0=1, dim1=2).contiguous().view(B, H, W, -1)
            y = self.out_norm(y)
            y = y * F.silu(z)

            setattr(self, "yz", y.view(B, H * W, -1).permute(0, 2, 1))
            
            out = self.out_proj(y)
            if self.dropout is not None:
                out = self.dropout(out)
            return out

    for layer in model.layers:
        for blk in layer.blocks:
            ss2d = blk.self_attention
            setattr(ss2d, "DEBUG", True)
            ss2d.forward = partial(ss2d_forward, ss2d)

    return model


ckpt = "/home/LiuYue/Workspace3/ckpts/vssmsmall/ema_ckpt_epoch_238.pth"
vssm = build_vssm(ckpt=ckpt, only_backbone=False, with_norm=True, depths=[2, 2, 27, 2], dims=96,)
vssm = add_hook(vssm)


data = build_dataset()
out = vssm(data[0][0])
print(out)




Successfully load ckpt /home/LiuYue/Workspace3/ckpts/vssmsmall/ema_ckpt_epoch_238.pth


NameError: name 'transforms' is not defined